imports

In [1]:
from tqdm.notebook import tqdm as tqdm
import os
import json
import pandas as pd

from HGNN.train.configParser import ConfigParser, getModelName, getDatasetName
from myhelpers import TrialStatistics

parameters

In [2]:
experimentsPath="/home/elhamod/HGNN/experiments/"
dataPath="/data/BGNN_data"
experimentName="learningRateDecay"

cuda=6

file and changes

In [3]:
old_params_file = os.path.join(experimentsPath, experimentName, "params.json")

# Keep same order as ConfigParserWriter!
changes = {
    "numOfTrials": 5
}

prepare

In [4]:
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
paramsIterator = config_parser.getExperiments()  
number_of_experiments = sum(1 for e in paramsIterator)

# populate a hash table of all experiments, indexed by trial hash
experiments = {}
for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
    num_of_models = experiment_params["numOfTrials"]
    for k in range(num_of_models):
        old_trialHash = TrialStatistics.getTrialName(experiment_params, k)
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)
        new_trialHash = TrialStatistics.getTrialName(experiment_params_new, k)
        experiments[old_trialHash] = {"trialHash": new_trialHash}

Fix data splits

In [5]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        # update experiment params
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)

        # fix data split names
        old_dataset_name = getDatasetName(config_parser.fixPaths(experiment_params))
        old_datasplitsPath = os.path.join(experimentsPath, experimentName, old_dataset_name)
        new_dataset_name = getDatasetName(config_parser.fixPaths(experiment_params_new))
        new_datasplitsPath = os.path.join(experimentsPath, experimentName, new_dataset_name)
        
        try:
            if os.path.exists(old_datasplitsPath):

                # update the folder names
                os.rename(old_datasplitsPath, new_datasplitsPath)
                print("datasplit", old_datasplitsPath, '->', new_datasplitsPath, "updated")

                # update the params.json
                j = json.dumps(experiment_params_new)
                json_fileName = os.path.join(new_datasplitsPath, 'params.json')
                f = open(json_fileName,"w")        
                f.write(j)
                f.close()  
                print("json", json_fileName, "updated")
                
            else:
                raise
                
        except:
            print("datasplit", old_dataset_name, "could not be fixed to", new_dataset_name)
            pass   
        
        experiments[TrialStatistics.getTrialName(experiment_params, 0)]['datasetName'] = new_dataset_name

        
        bar.update()

datasplit /home/elhamod/HGNN/experiments/learningRateDecay/datasplits/2788d2bc2c355b1c8cf2f3ac00958c413530a8f359f22067b2b8cfbc -> /home/elhamod/HGNN/experiments/learningRateDecay/datasplits/2788d2bc2c355b1c8cf2f3ac00958c413530a8f359f22067b2b8cfbc updated
json /home/elhamod/HGNN/experiments/learningRateDecay/datasplits/2788d2bc2c355b1c8cf2f3ac00958c413530a8f359f22067b2b8cfbc/params.json updated
datasplit /home/elhamod/HGNN/experiments/learningRateDecay/datasplits/2788d2bc2c355b1c8cf2f3ac00958c413530a8f359f22067b2b8cfbc -> /home/elhamod/HGNN/experiments/learningRateDecay/datasplits/2788d2bc2c355b1c8cf2f3ac00958c413530a8f359f22067b2b8cfbc updated
json /home/elhamod/HGNN/experiments/learningRateDecay/datasplits/2788d2bc2c355b1c8cf2f3ac00958c413530a8f359f22067b2b8cfbc/params.json updated
datasplit /home/elhamod/HGNN/experiments/learningRateDecay/datasplits/2788d2bc2c355b1c8cf2f3ac00958c413530a8f359f22067b2b8cfbc -> /home/elhamod/HGNN/experiments/learningRateDecay/datasplits/2788d2bc2c355b1c

Fix models

In [6]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)

        num_of_models = experiment_params["numOfTrials"]
        with tqdm(total=num_of_models, desc="trial") as bar2:
            for k in range(num_of_models):
            
                # Fix model names
                old_model_name = getModelName(experiment_params, k)
                old_modelPath = os.path.join(experimentsPath, experimentName, old_model_name)
                new_model_name = getModelName(experiment_params_new, k)
                new_modelPath = os.path.join(experimentsPath, experimentName, new_model_name)

                try:
                    if os.path.exists(old_modelPath):
                        os.rename(old_modelPath, new_modelPath)
                        print("model", old_modelPath, '->', new_modelPath, "updated")

                        j = json.dumps(experiment_params_new)
                        json_fileName = os.path.join(new_modelPath, 'params.json')
                        f = open(json_fileName,"w")        
                        f.write(j)
                        f.close()  
                        print("json", json_fileName, "updated")
                    else:
                        raise
                    
                except:
                    print("model", old_model_name, "could not be fixed to", new_model_name)
                    pass  
                
                

                bar2.update()
                
                experiments[TrialStatistics.getTrialName(experiment_params, k)]['modelName'] = new_model_name

        bar.update()

model /home/elhamod/HGNN/experiments/learningRateDecay/models/528b945e39ac36c5b753c4cd1782448c5ac1b787d9e40673d1188a36 -> /home/elhamod/HGNN/experiments/learningRateDecay/models/5b0ee470deceb7bf49e1f3ef1b4e11c9eb28a0e7547d6f70ef321161 updated
json /home/elhamod/HGNN/experiments/learningRateDecay/models/5b0ee470deceb7bf49e1f3ef1b4e11c9eb28a0e7547d6f70ef321161/params.json updated
model /home/elhamod/HGNN/experiments/learningRateDecay/models/5f1201f392ab8b1a11b2ff5e67d9b247cd3c4935a56d18fe94702682 -> /home/elhamod/HGNN/experiments/learningRateDecay/models/f50d9131cb91206e57e8f3910e33ddfe7d6f896cce51b24d5f08d8ca updated
json /home/elhamod/HGNN/experiments/learningRateDecay/models/f50d9131cb91206e57e8f3910e33ddfe7d6f896cce51b24d5f08d8ca/params.json updated
model /home/elhamod/HGNN/experiments/learningRateDecay/models/33001298a97c7c8c99988c583d0db3372baf5e19a1294f6c9ac361aa -> /home/elhamod/HGNN/experiments/learningRateDecay/models/a806689000bd4b5a2443d1567b98327ff519646d9fb3ec4cdfa581ff upd

model /home/elhamod/HGNN/experiments/learningRateDecay/models/6ebabb66d5df6e93590b955ce54420b82954faad8038e9a73a6d61a0 -> /home/elhamod/HGNN/experiments/learningRateDecay/models/cfebc02050936cb479082f1f0acca8cc266fc0175cffb57b91f99161 updated
json /home/elhamod/HGNN/experiments/learningRateDecay/models/cfebc02050936cb479082f1f0acca8cc266fc0175cffb57b91f99161/params.json updated
model /home/elhamod/HGNN/experiments/learningRateDecay/models/3716e87d39a02684469ecdc2ba2c49d914a09631c4e03e33cf36f343 -> /home/elhamod/HGNN/experiments/learningRateDecay/models/753383b2bbc0f0ea0f57fccc874ebc4025b7a0085036218dd886dcbe updated
json /home/elhamod/HGNN/experiments/learningRateDecay/models/753383b2bbc0f0ea0f57fccc874ebc4025b7a0085036218dd886dcbe/params.json updated
model /home/elhamod/HGNN/experiments/learningRateDecay/models/4682f72d1495bc72cd76e732a4cc1e83749ad2e78c67f6f0b5c0a753 -> /home/elhamod/HGNN/experiments/learningRateDecay/models/a3f2599605669f8dcb985d062efba2149817c34c104038e8d29e3f98 upd

model /home/elhamod/HGNN/experiments/learningRateDecay/models/184a4be739f6570b28cb120dfa831ff07f7c17d89b44db4d4e7b2078 -> /home/elhamod/HGNN/experiments/learningRateDecay/models/f7757e8ffc7802b85f32f1632c7f2c587af092d492a2e26809eef1b2 updated
json /home/elhamod/HGNN/experiments/learningRateDecay/models/f7757e8ffc7802b85f32f1632c7f2c587af092d492a2e26809eef1b2/params.json updated
model /home/elhamod/HGNN/experiments/learningRateDecay/models/47b50bef1cbad087832385acf3654e8da2c9bb819bb32da25b68fa6c -> /home/elhamod/HGNN/experiments/learningRateDecay/models/d74076fd22fa2e992868fca1a77b8a9a86912fcb9007b0a6894f7e2b updated
json /home/elhamod/HGNN/experiments/learningRateDecay/models/d74076fd22fa2e992868fca1a77b8a9a86912fcb9007b0a6894f7e2b/params.json updated
model /home/elhamod/HGNN/experiments/learningRateDecay/models/7687d943583e7b8dd7182a2610b75000fb789a97d11060470ef05755 -> /home/elhamod/HGNN/experiments/learningRateDecay/models/48f431d7ca224cd24717d291c42e8c0576fa2b81786c58b7a88e5679 upd

Fix results directory

In [7]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)

        old_experimentHash = TrialStatistics.getTrialName(experiment_params)
        old_experimentPath = os.path.join(experimentsPath, experimentName, "results", old_experimentHash)
        new_experimentHash = TrialStatistics.getTrialName(experiment_params_new)
        new_experimentPath = os.path.join(experimentsPath, experimentName, "results", new_experimentHash)
        
        try:
            if os.path.exists(old_experimentPath):

                os.rename(old_experimentPath, new_experimentPath)
                print("result", old_experimentPath, '->', new_experimentPath, "updated")

                j = json.dumps(experiment_params_new)
                json_fileName = os.path.join(new_experimentPath, 'params.json')
                f = open(json_fileName,"w")        
                f.write(j)
                f.close()  
                print("json", json_fileName, "updated")
            else:
                raise
                
        except:
            print("result", old_experimentHash, "could not be fixed to",new_experimentHash)
            pass 
        
        experiments[TrialStatistics.getTrialName(experiment_params, 0)]['experimentHash'] = new_experimentHash
        
        bar.update()

result /home/elhamod/HGNN/experiments/learningRateDecay/results/820717ed8f08d7dddf10f9771ce303f82ed59ba4de8a36cd646f15ac -> /home/elhamod/HGNN/experiments/learningRateDecay/results/4d9376173150019b4d3a8435fcb73f89e2bfaea7ebe1b626036e3cb8 updated
json /home/elhamod/HGNN/experiments/learningRateDecay/results/4d9376173150019b4d3a8435fcb73f89e2bfaea7ebe1b626036e3cb8/params.json updated
result /home/elhamod/HGNN/experiments/learningRateDecay/results/cebb458b28b820c1874142e88b397a51794965aa90a7a7534ac3d7aa -> /home/elhamod/HGNN/experiments/learningRateDecay/results/ca3bd4bcdc6067d4803682eddc63645349c6067162ae4576d53f39fd updated
json /home/elhamod/HGNN/experiments/learningRateDecay/results/ca3bd4bcdc6067d4803682eddc63645349c6067162ae4576d53f39fd/params.json updated
result /home/elhamod/HGNN/experiments/learningRateDecay/results/5b5b4cf2a9fdd8ec7c41edb417b20d106323552b3ac22d57380fcbee -> /home/elhamod/HGNN/experiments/learningRateDecay/results/7e71436c7c17b6caae8c52ba96db096fd88713fc8e18bdabc

Fix experiments file

In [8]:
# open experiments.csv file
experimentsFileNameAndPath=os.path.join(experimentsPath, "experiments.csv")
experiments_file= pd.read_csv(experimentsFileNameAndPath)

# For all experiments
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        numOfTrials = experiment_params["numOfTrials"]
        trialHash_0 = TrialStatistics.getTrialName(experiment_params, 0) # used for keys who don't change for different trials
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)
        
        # For all trials
        with tqdm(total=num_of_models, desc="trial") as bar2:
            for j in range(numOfTrials):
                
                trialHash = TrialStatistics.getTrialName(experiment_params, j)
                try:
                    # If old entry doesn't exist, nothing to update
                    record_exists = not (experiments_file[experiments_file['trialHash'] == trialHash][experiments_file['experimentName'] == experimentName]).empty
                    if record_exists:
                        
                        # drop old row and caluclate new row
                        new_trialHash = experiments[trialHash]["trialHash"]
                        experiments_file.drop(experiments_file[experiments_file['trialHash'] == trialHash][experiments_file['experimentName'] == experimentName].index, inplace = True) 
                        row_information = {
                            'experimentName': experimentName,
                            'modelName': experiments[trialHash]["modelName"],
                            'datasetName': experiments[trialHash_0]["datasetName"],
                            'experimentHash': experiments[trialHash_0]["experimentHash"],
                            'trialHash': new_trialHash
                        }
                        row_information = {**row_information, **experiment_params_new} 

                        # error handling to avoid duplicates
                        record_exists = not (experiments_file[experiments_file['trialHash'] == new_trialHash][experiments_file['experimentName'] == experimentName]).empty
                        if record_exists:
                            experiments_file.drop(experiments_file[experiments_file['trialHash'] == new_trialHash][experiments_file['experimentName'] == experimentName].index, inplace = True) 

                        # update with new entry    
                        experiments_file = experiments_file.append(pd.DataFrame(row_information, index=[0]), ignore_index = True)
                        print("trialHash", trialHash, "->", new_trialHash, "updated")
                except:
                    print("trialHash", trialHash, "could not be updated!")
                    pass
                bar2.update()
        bar.update()

# resave file
experiments_file.to_csv(experimentsFileNameAndPath, header=True, index=False)

trialHash 528b945e39ac36c5b753c4cd1782448c5ac1b787d9e40673d1188a36 -> 5b0ee470deceb7bf49e1f3ef1b4e11c9eb28a0e7547d6f70ef321161 updated
trialHash 5f1201f392ab8b1a11b2ff5e67d9b247cd3c4935a56d18fe94702682 -> f50d9131cb91206e57e8f3910e33ddfe7d6f896cce51b24d5f08d8ca updated
trialHash 33001298a97c7c8c99988c583d0db3372baf5e19a1294f6c9ac361aa -> a806689000bd4b5a2443d1567b98327ff519646d9fb3ec4cdfa581ff updated



/home/elhamod/melhamodenv3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/elhamod/melhamodenv3/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/elhamod/melhamodenv3/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


trialHash 6ebabb66d5df6e93590b955ce54420b82954faad8038e9a73a6d61a0 -> cfebc02050936cb479082f1f0acca8cc266fc0175cffb57b91f99161 updated
trialHash 3716e87d39a02684469ecdc2ba2c49d914a09631c4e03e33cf36f343 -> 753383b2bbc0f0ea0f57fccc874ebc4025b7a0085036218dd886dcbe updated
trialHash 4682f72d1495bc72cd76e732a4cc1e83749ad2e78c67f6f0b5c0a753 -> a3f2599605669f8dcb985d062efba2149817c34c104038e8d29e3f98 updated



trialHash 184a4be739f6570b28cb120dfa831ff07f7c17d89b44db4d4e7b2078 -> f7757e8ffc7802b85f32f1632c7f2c587af092d492a2e26809eef1b2 updated
trialHash 47b50bef1cbad087832385acf3654e8da2c9bb819bb32da25b68fa6c -> d74076fd22fa2e992868fca1a77b8a9a86912fcb9007b0a6894f7e2b updated
trialHash 7687d943583e7b8dd7182a2610b75000fb789a97d11060470ef05755 -> 48f431d7ca224cd24717d291c42e8c0576fa2b81786c58b7a88e5679 updated




Fix params file

In [9]:
experiment_list = []
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        experiment_params_new = {**experiment_params, **changes} 
        experiment_list.append(experiment_params_new)
        bar.update()


j = json.dumps({"experimentList": experiment_list})
f = open(old_params_file,"w")        
f.write(j)
f.close()  
print("json", old_params_file, "updated")


json /home/elhamod/HGNN/experiments/learningRateDecay/params.json updated
